In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
import json
from os.path import exists
from keras.applications import MobileNetV3Small

from numpy.random import choice as choose

import sys
sys.path.insert(0, "../Src/")
import basicUtil
import transferUtil

%autosave 5

Autosaving every 5 seconds


Autosaving every 5 seconds


Autosaving every 5 seconds


In [2]:
imageShape = (180,180)
inputShape = [imageShape[0],imageShape[1],3]
train = transferUtil.loadDataWithResizeShape("training",imageShape)
dev,test = transferUtil.loadDataWithResizeShape("testing",imageShape,.5)

base_model = MobileNetV3Small(
    include_top=False,
    classes=len(train.class_names),
    classifier_activation=None,
    input_shape=inputShape
)
for layer in base_model.layers:
    layer.trainable = False

possibleNeuronsPerLayer = basicUtil.createRangeFromMidpoint(16,32)
possibleEpochs = basicUtil.createRangeFromMidpoint(5,10)

dropoutCriticalPoints = (0,1)
adamLearningRateCriticalPoints = (1e-4,1e-2)
L2CriticalPoints = (1e-2,1e3) 

n_NeuronsPerFCLayers = []
n_Epochs = []
dropoutRates = []
adamLearningRates = []
L2Rates = []
trainScores = []
devScores = []

trial = 0
bestDevScore = 0
oversizedNeuronNumbers = []
oversizedEpochNumbers = []

while trial < 100:
    numberOfNeuronsPerFCLayer = choose(possibleNeuronsPerLayer)
    numberOfEpochs = choose(possibleEpochs)
    
    dropoutRate = basicUtil.generateDropoutRate(dropoutCriticalPoints[0],dropoutCriticalPoints[1])
    adamLearningRate = basicUtil.generateAdamLearningRate(adamLearningRateCriticalPoints[0],adamLearningRateCriticalPoints[1])
    L2Rate = basicUtil.generateL2(L2CriticalPoints[0],L2CriticalPoints[1])
    model = transferUtil.createTransferableModel(base_model, numberOfNeuronsPerFCLayer, dropoutRate, adamLearningRate, L2Rate)
    
    model.fit(train,epochs=numberOfEpochs,verbose=0)

    model_path = f'../Models/MobileNetTrials/mobilenet_model_{trial}.h5'
    model.save(model_path)
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    if model_size < 40:
        oversizedNeuronNumbers = []
        oversizedEpochNumbers = []
        print('trainScore')
        trainScore = model.evaluate(train)[1]
        print('devScore')
        devScore = model.evaluate(dev)[1]

        if (devScore > 0.91) and (devScore > bestDevScore):
            testScore = model.evaluate(test)[1]
            model_path = f'../Models/best_mobilenet_model_.h5'
            model.save(model_path)
            bestModelParams = {
                'n_NeuronsPerFCLayers' : int(numberOfNeuronsPerFCLayer),
                'n_Epochs' : int(numberOfEpochs),
                'dropoutRate' : dropoutRate,
                'adamLearningRates' : adamLearningRate,
                'L2Rates' : L2Rate,
                'modelSize' : model_size,
                'trainScore': trainScore,
                'devScore': devScore,
                'testScore': testScore
            }
            with open('../Models/best_mobilenet_model_params.json', 'w') as f:
                json.dump(bestModelParams, f)
            bestDevScore = devScore       

        n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
        n_Epochs.append(numberOfEpochs)
        
        adamLearningRates.append(adamLearningRate)
        dropoutRates.append(dropoutRate)
        L2Rates.append(L2Rate)
        trainScores.append(trainScore)
        devScores.append(devScore)
        
        print('concluding trial ',trial)
        trial += 1
    else:
        print(f'redoing trial {trial}. Model was {model_size}MB.')
        failedTrial = transferUtil.createTransferModelParametersDF([numberOfNeuronsPerFCLayer],[numberOfEpochs],
                                              [dropoutRate],[adamLearningRate],[L2Rate],[np.nan],[np.nan])
        display(failedTrial)
        oversizedNeuronNumbers.append(numberOfNeuronsPerFCLayer)
        oversizedEpochNumbers.append(numberOfEpochs)
        if len(oversizedNeuronNumbers) >= 3:
            possibleNeuronsPerLayer = [x for x in possibleNeuronsPerLayer if x < np.mean(oversizedNeuronNumbers)]
            possibleEpochs = [x for x in possibleEpochs if x < np.mean(oversizedEpochNumbers)]
            oversizedNeuronNumbers = []
            oversizedEpochNumbers = []
        
    if (trial % 10 == 9): 
        modelParametersDF = transferUtil.createTransferModelParametersDF(n_NeuronsPerFCLayers,n_Epochs,
                                                dropoutRates,adamLearningRates,L2Rates,trainScores,devScores)
        modelParametersDF = modelParametersDF.sort_values(by='trainScore', ascending=False)
        display(modelParametersDF)
        
        top5 = modelParametersDF[0:5]
        possibleNumberOfNeuronsPerFCLayer = basicUtil.getAdjustedRange(top5['n_NeuronsPerFCLayers'])
    
        possibleNumberOfEpochs = basicUtil.getAdjustedRange(top5['n_Epochs'])
        dropoutCriticalPoints = basicUtil.calculateCriticalPoints(top5['dropoutRate'])
        adamLearningRateCriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['adamLearningRates'])
        L2CriticalPoints = basicUtil.calculateLogisticCriticalPoints(top5['L2Rates'])

        n_NeuronsPerFCLayers = []
        n_Epochs = []
        dropoutRates = []
        adamLearningRates = []
        L2Rates = []
        trainScores = []
        devScores = []

        if bestDevScore > 0.91:
            trial = 101
            
basicUtil.displayFinalResults('../Models/best_mobilenet_model_params.json')
    

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-08-18 18:36:22.355637: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-08-18 18:36:22.355784: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2392]
	 [[{{node Placeholder/_0}}]]
2024-08-18 18:36:23.166767: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


trainScore
75/75 [==============================] - 9s 104ms/step - loss: 1.1632 - f1_score: 0.9507
devScore


2024-08-18 18:36:54.622509: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [299]
	 [[{{node Placeholder/_0}}]]
2024-08-18 18:36:54.622647: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 1s 73ms/step - loss: 1.1632 - f1_score: 0.9611


2024-08-18 18:36:55.779355: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [298]
	 [[{{node Placeholder/_4}}]]
2024-08-18 18:36:55.779462: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [298]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 1s 101ms/step - loss: 1.1633 - f1_score: 0.9360
concluding trial  0
trainScore
75/75 [==============================] - 9s 109ms/step - loss: 1.2834 - f1_score: 0.7172
devScore
10/10 [==============================] - 1s 78ms/step - loss: 1.3097 - f1_score: 0.6841
concluding trial  1
trainScore
75/75 [==============================] - 9s 107ms/step - loss: 1.0089 - f1_score: 0.9833
devScore
10/10 [==============================] - 1s 115ms/step - loss: 1.0107 - f1_score: 0.9942
concluding trial  2
trainScore
75/75 [==============================] - 8s 100ms/step - loss: 0.5146 - f1_score: 0.7246
devScore
10/10 [==============================] - 1s 78ms/step - loss: 0.5553 - f1_score: 0.7283
concluding trial  3
trainScore
75/75 [==============================] - 9s 106ms/step - loss: 0.6960 - f1_score: 0.9780
devScore
10/10 [==============================] - 1s 73ms/step - loss: 0.6953 - f1_score: 0.9780
concluding trial  4
trainScore
75/75 [====

,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,trainScore,devScore
2,22,1,0.699289,0.001287,0.023008,0.983324,0.989318
5,27,8,0.934928,0.000739,5.546534,0.978413,0.978684
4,25,7,0.607454,0.008716,645.952113,0.978009,0.978003
0,9,3,0.920272,0.000120,15.079694,0.950684,0.961105
8,4,7,0.717037,0.000135,0.208600,0.894881,0.860522
3,20,5,0.067152,0.000147,69.777538,0.724589,0.728349
1,7,1,0.230398,0.001344,5.360299,0.717235,0.684079
6,25,2,0.747232,0.000231,202.795486,0.550171,0.534843
7,7,1,0.619471,0.000132,153.814840,0.482614,0.469084


Final Model


,n_NeuronsPerFCLayers,n_Epochs,dropoutRate,adamLearningRates,L2Rates,modelSize,trainScore,devScore,testScore
0,22,1,0.699289,0.001287,0.023008,9.101433,0.983324,0.989318,0.994206
